In [14]:
import sys
import os
import tkinter as tk
import customtkinter as ctk
from tkinter import filedialog, messagebox
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException, WebDriverException
import pandas as pd
import time
import threading
from PIL import Image

# --- Variabel Global ---
file_path = ""
chrome_driver_path = ""

def resource_path(relative_path):
    try:
        base_path = sys._MEIPASS
    except Exception:
        base_path = os.path.abspath(".")
    return os.path.join(base_path, relative_path)

# --- Fungsi GUI ---
def select_file():
    global file_path
    file_path = filedialog.askopenfilename(title="Pilih File Excel", filetypes=[("Excel files", "*.xlsx *.xls"), ("All files", "*.*")])
    if file_path:
        file_name = file_path.split("/")[-1]
        file_label.configure(text=f"File Terpilih: {file_name}")
    else:
        file_label.configure(text="Belum ada file yang dipilih")

def select_driver_path():
    global chrome_driver_path
    chrome_driver_path = filedialog.askopenfilename(title="Pilih File ChromeDriver", filetypes=[("Executable files", "*.exe"), ("All files", "*.*")])
    if chrome_driver_path:
        driver_name = chrome_driver_path.split("/")[-1]
        driver_label.configure(text=f"Driver Manual: {driver_name}")
    else:
        driver_label.configure(text="Belum ada driver manual yang dipilih")

def update_log(message):
    try:
        log_textbox.configure(state="normal")
        log_textbox.insert("end", message + "\n")
        log_textbox.configure(state="disabled")
        log_textbox.see("end")
    except: pass

def update_progress(value):
    try: progress_bar.set(value)
    except: pass

def clear_log():
    log_textbox.configure(state="normal")
    log_textbox.delete("1.0", "end")
    log_textbox.configure(state="disabled")

def toggle_password():
    if show_password_var.get():
        password_entry.configure(show="")
    else:
        password_entry.configure(show="*")

# --- Fungsi Bantuan ---
def recover_and_re_navigate(driver, selected_rl):
    update_log("--> Error terdeteksi. Mencoba recovery dengan navigasi ulang...")
    try:
        update_log("--> Memaksa kembali ke halaman menu utama SIRS...")
        sirs_home_url = "https://akun-yankes.kemkes.go.id/beranda"
        driver.get(sirs_home_url)
        time.sleep(3) 

        if selected_rl == "RL 4.1":
            rl_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//a[@role='button' and text()='RL.4']")))
            driver.execute_script("arguments[0].click();", rl_element)
            time.sleep(1.5)
            dropdown_item = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(text(),'RL 4.1 Morbiditas Pasien Rawat Inap')]")))
            dropdown_item.click()
        elif selected_rl == "RL 5.1":
            rl_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//a[@role='button' and text()='RL.5']")))
            driver.execute_script("arguments[0].click();", rl_element)
            time.sleep(1.5)
            dropdown_item = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(text(),'RL 5.1 Mobiditas Pasien Rawat Jalan')]")))
            dropdown_item.click()
        
        time.sleep(2)
        button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, 'tambah')]")))
        driver.execute_script("arguments[0].click();", button)
        update_log("--> Recovery berhasil. Melanjutkan proses pada baris yang sama.")
    except Exception as e:
        update_log(f"(!) Gagal melakukan recovery. Error: {e}")
        raise e

def robust_clear_and_send_keys(driver, element_xpath, text_to_send):
    try:
        element = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, element_xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        time.sleep(0.5)
        element.click()
        element.send_keys(Keys.CONTROL + "a")
        element.send_keys(Keys.DELETE)
        time.sleep(0.5)
        element.send_keys(text_to_send)
    except Exception as e:
        update_log(f"(!) Gagal membersihkan atau mengirim teks ke elemen. Error: {e}")
        raise e

# --- FUNGSI UTAMA (CORE LOGIC) ---
def run_selenium_process():
    global chrome_driver_path
    driver = None
    
    # Variabel penampung (Aman dari crash)
    skipped_rows_log = [] 
    skipped_data_buffer = [] 
    should_save_excel = save_excel_var.get()
    
    try:
        email = email_entry.get()
        password = password_entry.get()
        selected_rl = rl_choice.get() 
        
        update_log("Membaca file Excel...")
        if selected_rl == "RL 5.1":
            df = pd.read_excel(file_path, header=4)
        else:
            df = pd.read_excel(file_path, header=3)
        total_rows = len(df)
        update_log(f"Ditemukan {total_rows} baris data untuk diproses.")
        
        try:
            update_log("Mencoba setup ChromeDriver otomatis...")
            service = ChromeService(ChromeDriverManager().install())
            driver = webdriver.Chrome(service=service)
        except Exception as e_auto:
            update_log(f"(!) Gagal setup otomatis: {e_auto}")
            if chrome_driver_path:
                update_log("Mencoba menggunakan driver manual...")
                service = ChromeService(executable_path=chrome_driver_path)
                driver = webdriver.Chrome(service=service)
            else:
                update_log("ERROR: Setup otomatis gagal, tidak ada driver manual dipilih.")
                start_button.configure(state="normal")
                return
        
        driver.get("https://akun-yankes.kemkes.go.id/beranda")
        driver.maximize_window()
        driver.implicitly_wait(15) 
        
        update_log("Melakukan login...")
        driver.find_element(By.ID, "floatingInput").send_keys(email)
        driver.find_element(By.ID, "floatingPassword").send_keys(password)
        time.sleep(1)
        driver.find_element(By.CLASS_NAME, "btn-outline-success").click()
        time.sleep(2)
        driver.find_element(By.XPATH, "//*[@id='root']/div/div[2]/a").click()
        driver.implicitly_wait(20)
        
        update_log("Login berhasil.")
        update_log(f"Memilih menu {selected_rl}...")
        
        if selected_rl == "RL 4.1":
            try:
                rl4_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@role='button' and text()='RL.4']")))
                driver.execute_script("arguments[0].click();", rl4_element)
                time.sleep(2)
                dropdown_item = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(text(),'RL 4.1 Morbiditas Pasien Rawat Inap')]")))
                dropdown_item.click()
            except:
                time.sleep(2)
                rl4_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@role='button' and text()='RL.4']")))
                driver.execute_script("arguments[0].click();", rl4_element)
                dropdown_item = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(text(),'RL 4.1 Morbiditas Pasien Rawat Inap')]")))
                dropdown_item.click()
        elif selected_rl == "RL 5.1":
            try:
                rl5_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@role='button' and text()='RL.5']")))
                driver.execute_script("arguments[0].click();", rl5_element)
                time.sleep(2)
                dropdown_item = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(text(),'RL 5.1 Mobiditas Pasien Rawat Jalan')]")))
                dropdown_item.click()
            except:
                time.sleep(2)
                rl5_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@role='button' and text()='RL.5']")))
                driver.execute_script("arguments[0].click();", rl5_element)
                dropdown_item = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(text(),'RL 5.1 Mobiditas Pasien Rawat Jalan')]")))
                dropdown_item.click()
                
        time.sleep(1)
        button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, 'tambah')]")))
        driver.execute_script("arguments[0].click();", button)

        month_mapping = {"January": 2, "February": 3, "March": 4, "April": 5, "May": 6, "June": 7, "July": 8, "August": 9, "September": 10, "October": 11, "November": 12, "December": 13}
        month_index = month_mapping[month_choice.get()]

        update_log("Memulai proses input data per baris...")
        
        # --- LOOPING UTAMA BARIS EXCEL ---
        for i, row in df.iterrows():

            # 1. VALIDASI MATEMATIKA
            if selected_rl == "RL 4.1":
                try:
                    def clean_val(val):
                        try: return int(float(str(val).replace(',', '.').strip()))
                        except: return 0
                    
                    icd_text = str(row.iloc[1]) 
                    nama_penyakit = str(row.iloc[2])

                    real_sum_L = 0
                    real_sum_P = 0
                    for j in range(25):
                        real_sum_L += clean_val(row.iloc[3 + 2 * j])
                        real_sum_P += clean_val(row.iloc[4 + 2 * j])

                    val_mati_L = clean_val(row.iloc[55])
                    val_mati_P = clean_val(row.iloc[56])

                    invalid_L = val_mati_L > real_sum_L
                    invalid_P = val_mati_P > real_sum_P

                    if invalid_L or invalid_P:
                        msg_list = []
                        if invalid_L: msg_list.append(f"Laki (Mati {val_mati_L} > Total Rincian {real_sum_L})")
                        if invalid_P: msg_list.append(f"Perempuan (Mati {val_mati_P} > Total Rincian {real_sum_P})")
                        full_msg = ", ".join(msg_list)
                        
                        update_log(f"(!) SKIP Baris {i+1} | ICD: {icd_text} | Jumlah Error -> {full_msg}")
                        skipped_rows_log.append(f"Baris {i+1} | ICD: {icd_text} | {full_msg}")
                        
                        row_error = row.copy()
                        row_error['KETERANGAN_ERROR'] = f"Jumlah Error: {full_msg}"
                        skipped_data_buffer.append(row_error)
                        
                        update_progress((i + 1) / total_rows)
                        continue 
                except Exception as e_check:
                    pass

            # 2. PROSES INPUT SELENIUM
            row_processed_successfully = False
            attempts_for_this_row = 0
            
            while not row_processed_successfully and attempts_for_this_row < 3:
                try:
                    update_log("-" * 20)
                    update_log(f"Memproses baris {i + 1}/{total_rows}")
                    
                    original_icd = str(row[1])
                    icd_data = original_icd 
                    # if '.' in icd_data and len(icd_data.split('.')) > 1 and len(icd_data.split('.')[1]) > 1:
                       # parts = icd_data.split('.')
                       # icd_data = f"{parts[0]}.{parts[1][0]}"
                    
                    update_log(f"({i+1}/{total_rows}) Mencari ICD: {icd_data}")
                    icd_input_xpath = "//input[@name='caripenyakit']"
                    
                    robust_clear_and_send_keys(driver, icd_input_xpath, icd_data)
                    driver.find_element(By.XPATH, icd_input_xpath).send_keys(Keys.RETURN)
                    
                    # [PENTING] Jeda agar tabel lama hilang dan tabel baru muncul
                    time.sleep(1.5)
                    
                    # --- LOGIKA SATPAM LOADING & TURBO (ZERO LATENCY) ---
                    try:
                        driver.implicitly_wait(0) # WAJIB 0
                        
                        xpath_loading = "//*[@id='root']/div/div[2]/div/div/div/div[1]/div[1]"
                        
                        # --- FASE 1: SATPAM LOADING ---
                        spinner_muncul = False
                        try:
                            WebDriverWait(driver, 0.5).until(EC.presence_of_element_located((By.XPATH, xpath_loading)))
                            spinner_muncul = True
                            update_log("    (i) Website sedang loading...")
                        except TimeoutException:
                            spinner_muncul = False

                        if spinner_muncul:
                            try:
                                WebDriverWait(driver, 30).until(EC.invisibility_of_element_located((By.XPATH, xpath_loading)))
                                update_log("    (i) Loading selesai.")
                            except TimeoutException:
                                update_log("(!) Loading macet > 30 detik. Reload...")
                                raise TimeoutException("Spinner Macet.")

                        # --- FASE 2: CEK DATA (TURBO MODE) ---
                        xpath_target = f"//table/tbody/tr[td[2][normalize-space()='{icd_data}']]//button"
                        target_ketemu = False
                        
                        start_time = time.time()
                        
                        while time.time() - start_time < 10:
                            
                            # A. Cek Exact Match
                            tombol_list = driver.find_elements(By.XPATH, xpath_target)
                            if len(tombol_list) > 0:
                                tombol = tombol_list[0]
                                update_log(f"    -> Match: {icd_data}")
                                driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'nearest'});", tombol)
                                time.sleep(0.5)
                                driver.execute_script("arguments[0].click();", tombol)
                                target_ketemu = True
                                break 
                            
                            # B. Cek "Data Tidak Ditemukan"
                            cek_kosong = driver.find_elements(By.XPATH, "//td[contains(text(), 'Data tidak ditemukan')]")
                            if len(cek_kosong) > 0:
                                msg = "Data Tidak Ditemukan (Valid)."
                                update_log(f"(!) SKIP Baris {i+1}: {msg}")
                                skipped_rows_log.append(f"Baris {i+1} | ICD: {icd_data} | {msg}")
                                
                                row_error = row.copy()
                                row_error['KETERANGAN_ERROR'] = msg
                                skipped_data_buffer.append(row_error)
                                
                                target_ketemu = "SKIP"
                                break

                            # C. Cek Ambigu
                            baris_lain = driver.find_elements(By.XPATH, "//*[@id='root']/div/div[2]/div/div/div/div[2]/table/tbody/tr")
                            if len(baris_lain) > 0:
                                teks_baris = baris_lain[0].text.strip()
                                if teks_baris and "Data tidak ditemukan" not in teks_baris:
                                    msg = f"Pilihan ICD lain muncul ({len(baris_lain)} opsi), tapi TIDAK ADA ICD yang sama persis."
                                    update_log(f"(!) SKIP Baris {i+1}: {msg}")
                                    skipped_rows_log.append(f"Baris {i+1} | ICD: {icd_data} | {msg}")
                                    
                                    row_error = row.copy()
                                    row_error['KETERANGAN_ERROR'] = msg
                                    skipped_data_buffer.append(row_error)
                                    
                                    target_ketemu = "SKIP"
                                    break 
                            
                            time.sleep(0.5)

                        # --- FASE 3: KEPUTUSAN FINAL ---
                        if target_ketemu == "SKIP":
                            driver.implicitly_wait(15) 
                            row_processed_successfully = True
                            break 

                        if not target_ketemu:
                            # Masih pakai wait 0 di sini
                            cek_baris = driver.find_elements(By.XPATH, "//*[@id='root']/div/div[2]/div/div/div/div[2]/table/tbody/tr")
                            
                            if len(cek_baris) == 0:
                                msg = "ICD Tidak Ditemukan."
                                update_log(f"(!) SKIP Baris {i+1}: {msg}")
                                skipped_rows_log.append(f"Baris {i+1} | ICD: {icd_data} | {msg}")
                                
                                row_error = row.copy()
                                row_error['KETERANGAN_ERROR'] = msg
                                skipped_data_buffer.append(row_error)
                                
                                driver.implicitly_wait(15) 
                                row_processed_successfully = True
                                break 
                            else:
                                driver.implicitly_wait(15) 
                                raise TimeoutException("Tabel error / target miss.")
                        
                        driver.implicitly_wait(15)

                    except Exception as e_poll:
                        driver.implicitly_wait(15) # Safety restore
                        raise e_poll 
                    # --- END LOGIKA ---


                    selected_year = year_choice.get() # Ambil dari GUI
                    try:
                        # 1. KLIK KOTAKNYA DULU (Biar terbuka)
                        driver.find_element(By.XPATH, "//*[@id='tahun']").click()
                        time.sleep(0.5) # Wajib jeda sedikit biar list turun
                        
                        # 2. BARU KLIK ANGKANYA
                        # Kita cari opsi yang teks-nya sesuai (2025 atau 2026)
                        driver.find_element(By.XPATH, f"//*[@id='tahun']/option[contains(text(), '{selected_year}')]").click()
                        
                        update_log(f"Berhasil memilih tahun: {selected_year}")

                    except Exception as e:
                        # JIKA KLIK GAGAL, PAKAI CARA 'KETIK' (Jalan Tikus)
                        update_log(f"(!) Klik gagal, mencoba mengetik tahun... {e}")
                        element_tahun = driver.find_element(By.XPATH, "//*[@id='tahun']")
                        element_tahun.send_keys(selected_year)
                        element_tahun.send_keys(Keys.ENTER)

                    # Lanjut Isi Form
                    month_dropdown_xpath = f"//*[@id='bulan']/option[{month_index}]"
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, month_dropdown_xpath))).click()

                    
                    for j in range(25):
                        male_data, female_data = row.iloc[3 + 2 * j], row.iloc[4 + 2 * j]
                        male_xpath = f"//*[@id='root']/div/div[2]/div[2]/div/div/form/div[2]/div[2]/table/tbody/tr[{j + 1}]/td[3]/input"
                        female_xpath = f"//*[@id='root']/div/div[2]/div[2]/div/div/form/div[2]/div[2]/table/tbody/tr[{j + 1}]/td[4]/input"
                        try:
                            male_element = driver.find_element(By.XPATH, male_xpath)
                            if male_element.is_enabled(): male_element.send_keys(str(male_data))
                        except: pass
                        try:
                            female_element = driver.find_element(By.XPATH, female_xpath)
                            if female_element.is_enabled(): female_element.send_keys(str(female_data))
                        except: pass
                        
                    male_last_idx, female_last_idx = (55, 56) if selected_rl == "RL 4.1" else (56, 57)
                    male_last_data, female_last_data = row.iloc[male_last_idx], row.iloc[female_last_idx]
                    try: driver.find_element(By.XPATH, "//*[@id='root']/div/div[2]/div[2]/div/div/form/div[2]/div[2]/table/tbody/tr[26]/td[3]/input").send_keys(str(male_last_data))
                    except: pass
                    try: driver.find_element(By.XPATH, "//*[@id='root']/div/div[2]/div[2]/div/div/form/div[2]/div[2]/table/tbody/tr[26]/td[4]/input").send_keys(str(female_last_data))
                    except: pass
                    
                    tombol_simpan = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Simpan')]")))
                    driver.execute_script("arguments[0].click();", tombol_simpan)
                    
                    # Klik Tambah untuk data berikutnya
                    button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, 'tambah')]")))
                    driver.execute_script("arguments[0].click();", button)
                    
                    row_processed_successfully = True
                    update_progress((i + 1) / total_rows)

                except (WebDriverException, NoSuchElementException, TimeoutException) as e:
                    # Cek jika error karena browser di-close
                    err_msg = str(e).lower()
                    if "no such window" in err_msg or "target window already closed" in err_msg or "disconnected" in err_msg:
                        # BREAK dari loop percobaan, yang akan otomatis BREAK dari loop utama karena error
                        # dan ditangkap di EXCEPT TERLUAR (di luar loop)
                        raise e 
                    
                    attempts_for_this_row += 1
                    update_log(f"(!) Gagal pada baris {i + 1}. Percobaan {attempts_for_this_row}/3. Error: {str(e)[:50]}")
                    if attempts_for_this_row < 3:
                        recover_and_re_navigate(driver, selected_rl)
                    else:
                        update_log(f"(!) Gagal total memproses baris {i + 1}.")
                        skipped_rows_log.append(f"Baris {i+1} | ICD: {icd_data} | Error Sistem")
                        
                        row_error = row.copy()
                        row_error['KETERANGAN_ERROR'] = f"Error Sistem: {str(e)}"
                        skipped_data_buffer.append(row_error)
                        break

    except Exception as e:
        # INI ADALAH JARING PENGAMAN UTAMA
        update_log(f"\n[INFO] Proses berhenti: {e}")
        update_log("Menyelamatkan data yang sudah didapat...")

    # --- FINALLY: BAGIAN INI AKAN SELALU DIJALANKAN (FINISH MAUPUN CRASH) ---
    finally:
        update_log("\n" + "="*30)
        update_log("   REKAPITULASI (AUTO SAVE)   ")
        update_log("="*30)
        
        # 1. Tampilkan di Log
        if len(skipped_rows_log) > 0:
            update_log(f"Total Data Bermasalah: {len(skipped_rows_log)}")
            for item in skipped_rows_log:
                update_log(f"• {item}")
        else:
            update_log("Tidak ada data bermasalah yang tercatat (atau proses belum dimulai).")

        # 2. SIMPAN EXCEL (PENTING)
        if should_save_excel and len(skipped_data_buffer) > 0:
            try:
                update_log("\n[INFO] Menyimpan file error ke Excel...")
                error_df = pd.DataFrame(skipped_data_buffer)
                
                dir_name = os.path.dirname(file_path)
                base_name = os.path.basename(file_path)
                name_without_ext = os.path.splitext(base_name)[0]
                error_file_path = os.path.join(dir_name, f"{name_without_ext}_DATA_ERROR.xlsx")
                
                error_df.to_excel(error_file_path, index=False)
                update_log(f"✅ FILE TERSIMPAN AMAN: {error_file_path}")
            except Exception as e_save:
                update_log(f"(!) Gagal menyimpan Excel Error: {e_save}")
        
        update_log("\n✓ PROSES SELESAI / DIHENTIKAN ✓")
        
        if driver: 
            try: driver.quit()
            except: pass
        start_button.configure(state="normal")

# --- GUI Setup ---
def start_process():
    if not file_path or not password_entry.get():
        if not file_path: file_label.configure(text="Harap pilih file Excel!", text_color="red")
        if not password_entry.get(): update_log("ERROR: Harap masukkan password Anda.")
        return
    
    start_button.configure(state="disabled")
    clear_log()
    update_progress(0)
    tab_view.set("Log")
    
    process_thread = threading.Thread(target=run_selenium_process)
    process_thread.daemon = True
    process_thread.start()

ctk.set_appearance_mode("Dark")
ctk.set_default_color_theme("blue")
root = ctk.CTk()
root.title("ALTAIR AUTOMATION SIRS v4.6")
root.geometry("600x860")

main_frame = ctk.CTkFrame(master=root)
main_frame.pack(pady=20, padx=20, fill="both", expand=True)

logo_path = resource_path("logoaltair.png")
try:
    logo_data = Image.open(logo_path)
    root.logo_image = ctk.CTkImage(dark_image=logo_data, light_image=logo_data, size=(150, 115))
    logo_label = ctk.CTkLabel(master=main_frame, text="", image=root.logo_image)
    logo_label.pack(pady=(20, 20), padx=20)
except: pass

tab_view = ctk.CTkTabview(master=main_frame)
tab_view.pack(padx=20, pady=10, fill="both", expand=True)
tab_utama = tab_view.add("Proses Utama")
tab_pengaturan = tab_view.add("Pengaturan")
tab_log = tab_view.add("Log")

scroll_content = ctk.CTkScrollableFrame(master=tab_utama, fg_color="transparent")
scroll_content.pack(fill="both", expand=True)

login_frame = ctk.CTkFrame(master=scroll_content)
login_frame.pack(pady=10, padx=10, fill="x")
ctk.CTkLabel(master=login_frame, text="Informasi Login", font=ctk.CTkFont(weight="bold")).pack(pady=(10, 5), padx=10, anchor="w")
email_entry = ctk.CTkEntry(master=login_frame)
email_entry.insert(0, "rsudulinprovkalsel@gmail.com")
email_entry.configure(state="disabled")
email_entry.pack(pady=5, padx=10, fill="x")
password_entry = ctk.CTkEntry(master=login_frame, placeholder_text="Masukkan Password Akun", show="*")
password_entry.pack(pady=(5, 5), padx=10, fill="x")
show_password_var = ctk.BooleanVar()
ctk.CTkCheckBox(master=login_frame, text="Tampilkan Password", variable=show_password_var, command=toggle_password).pack(pady=(0, 10), padx=10, anchor="w")

process_frame = ctk.CTkFrame(master=scroll_content)
process_frame.pack(pady=10, padx=10, fill="x")
ctk.CTkLabel(master=process_frame, text="Pengaturan Proses", font=ctk.CTkFont(weight="bold")).pack(pady=(10, 10), padx=10, anchor="w")
ctk.CTkButton(master=process_frame, text="Pilih File Excel", command=select_file).pack(pady=10, padx=10, fill="x")
file_label = ctk.CTkLabel(master=process_frame, text="Belum ada file", wraplength=350, justify="center")
file_label.pack(pady=(0, 10), padx=10)
ctk.CTkLabel(master=process_frame, text="Jenis RL").pack(pady=(10, 0), padx=10, anchor="w")
rl_choice = ctk.StringVar(value="RL 4.1")
ctk.CTkComboBox(master=process_frame, values=["RL 4.1", "RL 5.1"], variable=rl_choice).pack(pady=(0, 10), padx=10, fill="x")
ctk.CTkLabel(master=process_frame, text="Tahun Pelaporan").pack(pady=(5, 0), padx=10, anchor="w")
year_choice = ctk.StringVar(value="2025")
ctk.CTkComboBox(master=process_frame, values=["2025", "2026"], variable=year_choice).pack(pady=(0, 10), padx=10, fill="x")
ctk.CTkLabel(master=process_frame, text="Bulan Pelaporan").pack(pady=(10, 0), padx=10, anchor="w")
month_choice = ctk.StringVar(value="January")
ctk.CTkComboBox(master=process_frame, values=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"], variable=month_choice).pack(pady=(0, 10), padx=10, fill="x")

# --- TAMBAHAN PILIHAN EXCEL ---
save_excel_var = ctk.BooleanVar(value=True) 
ctk.CTkCheckBox(master=process_frame, text="Simpan Data Error ke Excel Otomatis", variable=save_excel_var).pack(pady=(5, 10), padx=10, anchor="w")

start_button = ctk.CTkButton(master=tab_utama, text="MULAI PROSES", command=start_process, height=40, font=ctk.CTkFont(size=14, weight="bold"))
start_button.pack(pady=20, padx=100, fill="x", side="bottom")

driver_frame = ctk.CTkFrame(master=tab_pengaturan)
driver_frame.pack(pady=10, padx=10, fill="both", expand=True)
ctk.CTkLabel(master=driver_frame, text="Pengaturan Driver (Cadangan)", font=ctk.CTkFont(weight="bold")).pack(pady=(10, 10), padx=10, anchor="w")
ctk.CTkLabel(master=driver_frame, text="Gunakan ini HANYA jika setup driver otomatis gagal.", wraplength=400, justify="left").pack(pady=(0, 10), padx=10, anchor="w")
ctk.CTkButton(master=driver_frame, text="Pilih Path Driver Manual", command=select_driver_path).pack(pady=10, padx=10, fill="x")
driver_label = ctk.CTkLabel(master=driver_frame, text="Belum ada driver manual", wraplength=350, justify="center")
driver_label.pack(pady=(0, 15), padx=10)

status_frame = ctk.CTkFrame(master=tab_log)
status_frame.pack(pady=10, padx=10, fill="both", expand=True)
progress_container = ctk.CTkFrame(master=status_frame, fg_color="transparent")
progress_container.pack(pady=10, padx=10, fill="x")
progress_bar = ctk.CTkProgressBar(master=progress_container)
progress_bar.set(0)
progress_bar.pack(side="left", fill="x", expand=True, padx=(0, 10))
ctk.CTkButton(master=progress_container, text="Hapus Log", command=clear_log, width=100).pack(side="left")
log_textbox = ctk.CTkTextbox(master=status_frame, state="disabled", wrap="word")
log_textbox.pack(pady=(0, 10), padx=10, fill="both", expand=True)

ctk.CTkLabel(master=main_frame, text="Developed by: Aldy Pradana, A.Md.RMIK | v4.6 (Stable Safe Mode)", font=ctk.CTkFont(size=10)).pack(pady=(10, 5), padx=20, side="bottom", anchor="e")

root.mainloop()